# SPR 2026 - Mistral 7B Instruct (4-bit Quantization)

## ✅ Compatível com competições que exigem Internet OFF

**Este notebook usa Mistral 7B Instruct com quantização 4-bit.**

### Configuração:

1. **NÃO precisa de internet** - funciona offline
2. Adicione o modelo: `mistralai/Mistral-7B-Instruct-v0.3` ou similar
3. Settings → Accelerator → **GPU T4 x2** ou **P100**
4. Run All

### Vantagens:
- **7B parâmetros** - modelo maior que 3-4B
- **4-bit quant** - ~10GB VRAM (cabe no T4)
- **Mistral AI** - excelente performance

### ⚠️ Nota sobre bitsandbytes:
O Kaggle já tem bitsandbytes instalado. Se der erro, rode sem quantização.

---

In [ ]:
# =============================================================================
# SPR 2026 - MISTRAL 7B INSTRUCT 4-BIT (OFFLINE)
# =============================================================================
# REQUER: Modelo "mistralai/Mistral-7B-Instruct-v0.3" adicionado
# =============================================================================

import os
import numpy as np
import pandas as pd
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

print("="*60)
print("SPR 2026 - Mistral 7B Instruct 4-bit (OFFLINE)")
print("="*60)

# ==== CONFIGURAÇÕES ====
SEED = 42
DATA_DIR = '/kaggle/input/spr-2026-mammography-report-classification'
USE_4BIT = True  # Usar quantização 4-bit

# Paths para modelos Kaggle
MODEL_PATHS = [
    '/kaggle/input/mistral-7b-instruct-v0.3/transformers/default/1',
    '/kaggle/input/mistral-7b-instruct/transformers/v0.3/1',
    '/kaggle/input/mistral/transformers/7b-instruct-v0.3/1',
    '/kaggle/input/mistral-7b/transformers/instruct/1',
]

MODEL_PATH = None
for path in MODEL_PATHS:
    if os.path.exists(path):
        MODEL_PATH = path
        break

if MODEL_PATH is None:
    print("\n⚠️ Modelo não encontrado. Datasets disponíveis:")
    for item in os.listdir('/kaggle/input'):
        print(f"  - {item}")
        try:
            for sub in os.listdir(f'/kaggle/input/{item}')[:3]:
                print(f"      └── {sub}")
        except:
            pass
    raise FileNotFoundError("Adicione o modelo mistralai/Mistral-7B-Instruct ao notebook!")

USE_GPU = torch.cuda.is_available()
device = 'cuda' if USE_GPU else 'cpu'

print(f"✓ Modelo: {MODEL_PATH}")
print(f"✓ GPU disponível: {USE_GPU}")
print(f"✓ Quantização 4-bit: {USE_4BIT}")

In [ ]:
# ==== CARREGAR DADOS ====
print("\n[1/4] Carregando dados...")
train = pd.read_csv(f'{DATA_DIR}/train.csv')
test = pd.read_csv(f'{DATA_DIR}/test.csv')
print(f"    Train: {train.shape} | Test: {test.shape}")

print("\nDistribuição das classes no treino:")
print(train['target'].value_counts().sort_index())

In [ ]:
# ==== CARREGAR MODELO ====
print("\n[2/4] Carregando Mistral 7B Instruct...")

tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)

if USE_4BIT and USE_GPU:
    # Configuração 4-bit para caber no T4 (16GB)
    quantization_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_quant_type="nf4",
    )
    
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        quantization_config=quantization_config,
        device_map='auto',
        trust_remote_code=True,
    )
    print("    ✓ Modelo carregado em 4-bit!")
    print(f"    ✓ VRAM estimada: ~10GB")
else:
    # Fallback: float16 sem quantização
    model = AutoModelForCausalLM.from_pretrained(
        MODEL_PATH,
        torch_dtype=torch.float16 if USE_GPU else torch.float32,
        device_map='auto' if USE_GPU else None,
        trust_remote_code=True,
    )
    if not USE_GPU:
        model = model.to(device)
    print("    ✓ Modelo carregado em float16!")

model.eval()

In [ ]:
# ==== PROMPT TEMPLATE (Mistral format) ====
# Mistral usa formato [INST] ... [/INST]

def build_prompt(report):
    return f"""[INST] Você é um radiologista especialista em mamografia.
Classifique o seguinte laudo de mamografia na categoria BI-RADS correta.

Categorias BI-RADS:
0 - Inconclusivo, necessita exames adicionais
1 - Negativo, sem achados
2 - Achados benignos
3 - Provavelmente benigno
4 - Suspeito de malignidade
5 - Altamente suspeito de malignidade
6 - Malignidade confirmada por biópsia

Laudo:
{report[:2000]}

Responda APENAS com o número da categoria BI-RADS (0, 1, 2, 3, 4, 5 ou 6): [/INST]"""

def classify_report(report, max_length=1024):
    """Classifica um laudo usando o LLM."""
    prompt = build_prompt(report)
    
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True, max_length=max_length)
    inputs = {k: v.to(device) for k, v in inputs.items()}
    
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=5,
            temperature=0.1,
            do_sample=False,
            pad_token_id=tokenizer.eos_token_id,
        )
    
    response = tokenizer.decode(outputs[0][inputs['input_ids'].shape[1]:], skip_special_tokens=True)
    
    # Extrair número da resposta
    for char in response.strip():
        if char.isdigit() and int(char) in range(7):
            return int(char)
    
    return 2  # Fallback

In [ ]:
# ==== INFERÊNCIA ====
print("\n[3/4] Classificando laudos do teste...")

predictions = []
for idx, row in tqdm(test.iterrows(), total=len(test), desc="Classificando"):
    pred = classify_report(row['text'])
    predictions.append(pred)

test['target'] = predictions

print("\nDistribuição das predições:")
print(test['target'].value_counts().sort_index())

In [ ]:
# ==== SALVAR SUBMISSION ====
print("\n[4/4] Salvando submission...")

submission = test[['id', 'target']].copy()
submission.to_csv('submission.csv', index=False)

print("\n" + "="*60)
print("✅ SUBMISSION SALVA: submission.csv")
print("="*60)
print(submission.head(10))